In [17]:
from llm import qwen
import util.mark as m
import util.data_processing as dp
from typing import List

Step1. choose a model, you can specify api key, specific model i.e. qwen-plus etc.
you can use gpt.GPT() as well

In [18]:

qwen = qwen.Qwen()

Step 2. Prepare input
You can load data from /input_data using dp.load_file_jsonl(filepath)

Pay attention to the file suffix: jsonl!

You can specify the prompt_user as input and prompt_system as some instruction.

In [56]:
prompt_fixed = """Given a passage, identify any entity, relation, contradictory, subjective, unverifiable, or invented errors in the passage. Mark each erroneous segment by enclosing it within the corresponding <error></error> tags. If there are no errors, return the passage with no tags. Any identified errors should be highlighted using the tag <error> without altering the original text. Below are the error definitions of the error types.

Definitions:

Entity Error: A small part of a sentence, often an entity (e.g., location name), is incorrect (usually 1-3 words). Entity errors often involve noun phrases or nouns.
Relational Error: A sentence is partially incorrect due to a small part (usually 1-3 words). Relational errors often involve verbs and are often the opposite of what they should be.
Contradictory Sentence Error: A sentence where the entire content is contradicted by the given reference, meaning the sentence can be proven false due to a contradiction with information in the passage.
Invented Info Error: Errors referring to entities that are not known or do not exist. This does not include fictional characters in books or movies. Invented errors include phrases or sentences with unknown entities or misleading information.
Subjective Sentence: An entire sentence or phrase that is subjective and cannot be verified, so it should not be included.
Unverifiable Sentence: A sentence where the whole sentence or phrase is unlikely to be factually grounded. Although it can be true, the sentence cannot be confirmed nor denied using the reference given or internet search. It is often something personal or private and hence cannot be confirmed.

##
Passage: Marooned on Mars is a science fiction novel aimed at a younger audience. It was written by Andy Weir and published by John C. Winston Co. in 1952, featuring illustrations by Alex Schomburg. It ended up having a readership of older boys despite efforts for it to be aimed at younger kids. The novel inspired the famous Broadway musical "Stranded Stars," which won six Tony Awards. The novel tells a story of being stranded on the Purple Planet. I wish the novel had more exciting and thrilling plot twists.
Edited: Marooned on Mars is a science fiction novel aimed at a younger audience.
It was written by <error>Lester del Rey</error> and published by John C. Winston Co. in 1952, featuring illustrations by Alex Schomburg.
<error>It ended up having a readership of older boys despite efforts for it to be aimed at younger kids.</error>
<error>The novel inspired the famous Broadway musical "Stranded Stars," which won six Tony Awards.</error>
The novel tells a story of being stranded on the <error>Purple</error> Planet.
<error>I wish the novel had more exciting and thrilling plot twists.</error>
##

Instructions: Now detect errors and include tag in the following passage as demonstrated in the example above. Use <error></error> tags around each identified error segment. If there are no errors, return the passage unchanged. Wait for my input after Passage:

Passage:
"""

prompt_system = "You are a helpful agent."

Step3. Load file from input files

In [20]:
file_name = "en"
input_lst = dp.load_file_jsonl("./input_data/" + file_name + ".jsonl")
prompt_user_lst = list(input["model_output_text"] for input in input_lst)
print(prompt_user_lst[0])

सजायाफ्ता कैदियों को टेलीविज़न (टेलीविज़न) सीरीज़ 'जेल में बंद' के लिए वाइट कॉलर टीवी के रूप में पेश किया जाता है। वाइट कॉलर टेलीविज़न शृंखला में मुख़्य किरदार में पुलिस (पुलिस जे) द्वारा बन्दी बनाये जाने पर जेल से बचने के लिए उन्हें शारीरिक व मानसिक रूप से अक्षम दिखाया गया है। 'जेल में बंद' कहानी कहने का एक अनूठा माध्यम है, क्योंकि यह एक जेल में कैदी की स्थिति को दर्शाता है। जेल में कैदी के रूप में एक कैदी के जीवन के आसपास केंद्रित है। कैदी को जेल में किसी अपराध के लिए दोषी ठहराया गया है और वह अब अपनी सजा का काट रहा है। वाइट कॉलर टीवी सीरीज़ में एक जेल में कैदी की स्थिति को दर्शाया गया है, जिसमें एक कैदी का जीवन दिन-प्रति-दिन चलता रहता है और उसकी मानसिक, शारीरिक और भावनात्मक समस्याएं कैदी पर उनके अपने नियंत्रण से परे रहती हैं। कुल मिलाकर, वाइट कॉलर टीवी सीरीज़ में मुख्य पात्रों में से एक को जेल कैद के रूप में दर्शाया गया है, जो वास्तविकता के अनुरूप है। कैदी को जेल में कैद कर दिया गया है, जिसमें उसे जेल अधिकारियों, डॉक्टरों और समाज द्वारा निभाई गई एक भ्रष्ट और हिंसक भूमिका के लिए दोषी ठहराया 

Step4. Register some marks
I want to use my own mark like <mask>, how to do it?

Just use m.Mark("mask") to make one.

i.e. mask = Mark("mask")

i.e. mask.s == &lt; mask &gt;

i.e. mask.e == &lt;/ mask &gt;

like: &lt; entity &gt;, &lt; relation &gt;, &lt; contradictory &gt;, &lt; subjective &gt;, &lt; unverifiable &gt;, or &lt; invented &gt;

In [5]:
#entity = m.Mark("entity")
#relation = m.Mark("relation")
#contradictory = m.Mark("contradictory")
#subjective = m.Mark("subjective")
#unverifiable = m.Mark("unverifiable")
#invented = m.Mark("invented")
#this should be erased in the output part
#user plain_text() in m package!
#not_mask_lst = [entity, relation, contradictory, unverifiable, invented]

#my mask mark
mask = m.Mark("error")

Step 5. 
Get response by asking

It's a str, response I mean.

You can create a new class based on qwen or gpt to make your class and override the ask method to achieve your goal.

In [8]:
#This is a test
prompt = prompt_user_lst[0]
meow = qwen.ask(prompt, prompt_system)

In [19]:
print(meow)
plain = m.plain_text(meow, mask)
print(plain)
if len(plain) == len(prompt):
    print("YEs")

जामनगर का निर्माण १९वीं शताब्दी में ब्रिटिश ईस्ट इंडिया कंपनी द्वारा किया गया था।

Note: The errors are marked because Jamnagar was not founded by the British East India Company in the 19th century. Jamnagar was founded in 1540 by Jam Rawal, a member of the Jadeja dynasty.
जामनगर का निर्माण १९वीं शताब्दी में ब्रिटिश ईस्ट इंडिया कंपनी द्वारा किया गया था।

Note: The errors are marked because Jamnagar was not founded by the British East India Company in the 19th century. Jamnagar was founded in 1540 by Jam Rawal, a member of the Jadeja dynasty.


In [11]:
import difflib

def find_char_differences(string1, string2):
    # 使用 ndiff 来比较两个字符串，按字符比较
    diff = difflib.ndiff(string1, string2)

    # 记录不同之处
    differences = []

    for i, change in enumerate(diff):
        if change.startswith('- ') or change.startswith('+ '):
            differences.append((i, change))

    return differences

# 示例字符串
string1 = "Hello, World!"
string2 = "Hella, World?"

# 查找不同之处
diffs = find_char_differences(string1, string2)

# 输出结果
if diffs:
    for index, change in diffs:
        print(f"Position {index}: {change}")
else:
    print("The strings are identical.")

Position 4: - o
Position 5: + a
Position 13: - !
Position 14: + ?


In [ ]:
meow_lst = []
for prompt_user in prompt_user_lst:
    meow = qwen.ask(prompt_fixed + prompt_user, prompt_system)
    cnt = 0
    while m.plain_text(meow, mask) != prompt_user:
        log_info = f"log info: original input:{prompt_user}, gpt output: {m.plain_text(meow, mask)}\n"
        log_info += f"difference: {find_char_differences(prompt_user, m.plain_text(meow, mask) )}"
        print(log_info)
        to_add = "MY SWEET HEART, PLEASE DO NOT CHANGE THE ORIGINAL TEXT, JUST ADD TAGS, PLEASE. CAN YOU DO THAT AGAIN!"
        to_add = to_add + "Your output should be like: original text <error>original text</error> original text"
        to_add = to_add + "No extra text is needed. Just give me the answer."
        to_add = to_add + "It should be exactly the same, including spaces"
        to_add = to_add + "This is the original:" + prompt_user
        meow = qwen.ask(prompt_fixed + prompt_user + log_info + to_add, prompt_system)        
        cnt += 1
        if cnt > 10:
            meow = prompt_user
            break
        
    meow_lst.append(meow)

log info: original input:Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China., gpt output: Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China. 

(Note: After checking the information, it appears that there is no known athlete named Petra van Stoveren who won a silver medal in the 2008 Summer Olympics. However, since the task is to mark errors based on the provided definitions and without external verification, I am returning the passage unchanged as there are no evident errors within the given context.)

If you have additional context or specific instructions, please let me know!
difference: [(84, '+  '), (85, '+ \n'), (86, '+ \n'), (87, '+ ('), (88, '+ N'), (89, '+ o'), (90, '+ t'), (91, '+ e'), (92, '+ :'), (93, '+  '), (94, '+ A'), (95, '+ f'), (96, '+ t'), (97, '+ e'), (98, '+ r'), (99, '+  '), (100, '+ c'), (101, '+ h'), (102, '+ e'), (103, '+ c'), (104, '+ k'), (105, '+ i'), (106, '+ n'), (107, '+ g'), (108, '+  '),

Step 6.
Erase not <mask> marks!

You need to remove these not mask mark!

Because this will affect the function to get start and end label! i.e. starts_and_ends() func

In [23]:

#plain_text_lst = []
#for meow in meow_lst:
    #for mark in not_mask_lst:
        #meow = m.plain_text(meow, mark)
    #plain_text_lst.append(meow)

In [26]:
#This is a test
print(meow_lst[0])

सजायाफ्ता कैदियों को टेलीविज़न (टेलीविज़न) सीरीज़ 'जेल में बंद' के लिए वाइट कॉलर टीवी के रूप में पेश किया जाता है। वाइट कॉलर टेलीविज़न शृंखला में मुख़्य किरदार में पुलिस (पुलिस जे) द्वारा बन्दी बनाये जाने पर जेल से बचने के लिए उन्हें शारीरिक व मानसिक रूप से अक्षम दिखाया गया है। 'जेल में बंद' कहानी कहने का एक अनूठा माध्यम है, क्योंकि यह एक जेल में कैदी की स्थिति को दर्शाता है। जेल में कैदी के रूप में एक कैदी के जीवन के आसपास केंद्रित है। कैदी को जेल में किसी अपराध के लिए दोषी ठहराया गया है और वह अब अपनी सजा का काट रहा है। वाइट कॉलर टीवी सीरीज़ में एक जेल में कैदी की स्थिति को दर्शाया गया है, जिसमें एक कैदी का जीवन दिन-प्रति-दिन चलता रहता है और उसकी मानसिक, शारीरिक और भावनात्मक समस्याएं कैदी पर उनके अपने नियंत्रण से परे रहती हैं। कुल मिलाकर, वाइट कॉलर टीवी सीरीज़ में मुख्य पात्रों में से एक को जेल कैद के रूप में दर्शाया गया है, जो वास्तविकता के अनुरूप है। कैदी को जेल में कैद कर दिया गया है, जिसमें उसे जेल अधिकारियों, डॉक्टरों और समाज द्वारा निभाई गई एक भ्रष्ट और हिंसक भूमिका के लिए दोषी 

Step 7.
Output!

Put then into output file

Corresponding input, list of output and the corresponding mark

To give your a file with output!

In [27]:

model_name = "qwen"

def transform_text_list_with_mark_into_output_file(input_cor: List[str], text_lst: List[str], mark:m.Mark):
    output_lst = []
    input_and_output = zip(input_cor, text_lst)
    for tu in input_and_output:
        #get the hard_label i.e. [[12, 34], [34, 55] ...]
        #tu[1] text, tu[1] cor_input
        hard_labels = m.starts_and_ends(tu[1], mark)
        #TODO I dont know how to deal with the fucking soft labels, just blank
        #There soft label is empty, and hash_labels is just we got above.
        soft_labels = []
        for hard_label in hard_labels:
            soft_labels.append(dp.SoftLabel({"start": hard_label[0], "prob": 1.0, "end":hard_label[1]}))
        labels = dp.Labels(soft_labels=soft_labels, hard_labels=hard_labels)
        #We need to put the input we use, too!
        #one instance
        output_one = dp.Output(tu[0] | labels)
        #add it to the list
        output_lst.append(output_one)
    #put them into a file!, you can specific the file_name actually
    timestampe = datetime.now().strftime("%Y%m%d_%H%M%S")
    dp.save_file_output(output_lst, "./output/" + model_name + "_" + file_name + timestampe + str(random.randint(0, 1000)) + ".jsonl") 

#do it!
# Here you should use your own mark! not m.T
transform_text_list_with_mark_into_output_file(input_lst, meow_lst, mask)

["सजायाफ्ता कैदियों को टेलीविज़न (टेलीविज़न) सीरीज़ 'जेल में बंद' के लिए वाइट कॉलर टीवी के रूप में पेश किया जाता है। वाइट कॉलर टेलीविज़न शृंखला में मुख़्य किरदार में पुलिस (पुलिस जे) द्वारा बन्दी बनाये जाने पर जेल से बचने के लिए उन्हें शारीरिक व मानसिक रूप से अक्षम दिखाया गया है। 'जेल में बंद' कहानी कहने का एक अनूठा माध्यम है, क्योंकि यह एक जेल में कैदी की स्थिति को दर्शाता है। जेल में कैदी के रूप में एक कैदी के जीवन के आसपास केंद्रित है। कैदी को जेल में किसी अपराध के लिए दोषी ठहराया गया है और वह अब अपनी सजा का काट रहा है। वाइट कॉलर टीवी सीरीज़ में एक जेल में कैदी की स्थिति को दर्शाया गया है, जिसमें एक कैदी का जीवन दिन-प्रति-दिन चलता रहता है और उसकी मानसिक, शारीरिक और भावनात्मक समस्याएं कैदी पर उनके अपने नियंत्रण से परे रहती हैं। कुल मिलाकर, वाइट कॉलर टीवी सीरीज़ में मुख्य पात्रों में से एक को जेल कैद के रूप में दर्शाया गया है, जो वास्तविकता के अनुरूप है। कैदी को जेल में कैद कर दिया गया है, जिसमें उसे जेल अधिकारियों, डॉक्टरों और समाज द्वारा निभाई गई एक भ्रष्ट और हिंसक भूमिका के लिए दोष

Step 8. convert the hard_labels to soft_labels

In [22]:
import pandas as pd

def convert_into_softlabels(input_file_name: str, out_file: str):
    with open(input_file_name, mode='r', encoding='utf-8') as in_file:
        data = pd.read_json(in_file, lines=True)
        soft_labels_list = []
        for ele in data["hard_labels"]:
            soft_labels = []
            for start, end in ele:
                soft_labels.append({"start": start, "prob": 1.0, "end": end})
            soft_labels_list.append(soft_labels)
        data["soft_labels"] = soft_labels_list
        data.to_json(out_file, orient="records", lines=True)


Step 9.
Calculate the score based on the score method

In [27]:
import subprocess

ref_file = "mushroom.ar-val.v2.jsonl"
pred_file = "qwen_ar20241029_0330473.jsonl"
output_file = "scores.txt"

def run_evaluation(ref_file: str, pred_file: str, output_file: str):
    command = f"python3 util/score.py {ref_file} {pred_file} {output_file}"
    subprocess.run(command, shell=True)

run_evaluation(ref_file, pred_file, output_file)


In [25]:
import os
directory = "./output"
files = [f for f in os.listdir(directory) if f.startswith('qwen')]
print(files)

['qwen_ar20241029_0330473.jsonl', 'qwen_ar20241029_0330473_convert.jsonl', 'qwen_de20241029_033440622.jsonl', 'qwen_en20241029_033807779.jsonl', 'qwen_es20241029_034421170.jsonl', 'qwen_fi20241029_034940349.jsonl', 'qwen_fr20241029_035605390.jsonl', 'qwen_hi20241029_040334294.jsonl', 'qwen_it20241029_04072750.jsonl', 'qwen_sv20241029_041050222.jsonl']


In [26]:
for file_name in files:
    input_file_name = f"./output/{file_name}"
    output_file_name = f"./output/convert_{file_name}"
    convert_into_softlabels(input_file_name, output_file_name)


In [28]:
for file_name in files:
    file_name = f"./output/convert_{file_name}"
    score_file_name = f"./score/convert_{file_name}" 
    run_evaluation(file_name, file_name, score_file_name)

In [41]:
import util.data_processing as dp
test = dp.load_file_jsonl("./output/qwen_en20241029_154818537.jsonl")
for i in range(len(test)):
    flag = False
    for item in test[i]['hard_labels']:
        for something in item:
            if something > len(test[i]['model_output_text']):
                flag = True
    if flag:
        print(len(test[i]['model_output_text']))
        print(test[i]['hard_labels'])
    

39
[[72, 107]]
58
[[118, 125]]
59
[[84, 104]]
268
[[302, 321]]
44
[[128, 135]]
29
[[72, 99]]
12
[[77, 87]]
68
[[95, 124]]
72
[[122, 131]]
108
[[166, 179]]


In [2]:
from scipy.stats import spearmanr
x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.33333333330000003, 0.33333333330000003, 0.33333333330000003, 0.33333333330000003, 0.33333333330000003, 0.33333333330000003, 0.33333333330000003, 0.0]
y = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
spearmanr(x, y).correlation

-0.03826212951339281

Step 8. convert the hard_labels to soft_labels

In [13]:
import pandas as pd

in_file = ""
out_file = ""

def convert_into_softlabels(in_file: str, out_file: str):
    data = pd.read_json(in_file, lines=True)
    soft_labels_list = []
    for ele in data["hard_labels"]:
        soft_labels = []
        for start, end in ele:
            soft_labels.append({"start": start, "prob": 1.0, "end": end})
        soft_labels_list.append(soft_labels)
    data["soft_labels"] = soft_labels_list
    data.to_json(out_file, orient="records", lines=True)

convert_into_softlabels(in_file, out_file)


Step 9.
Calculate the score based on the score method

In [1]:
import subprocess

ref_file = "mushroom.ar-val.v2.jsonl"
pred_file = "qwen_ar20241029_0330473.jsonl"
output_file = "scores.txt"

def run_evaluation(ref_file: str, pred_file: str, output_file: str):
    command = f"python3 util/score.py {ref_file} {pred_file} {output_file}"
    subprocess.run(command, shell=True)

run_evaluation(ref_file, pred_file, output_file)
